In [1]:
import pandas as pd
import glob
from pathlib import Path

def merge_csv_files(folder_path, output_file='merged_kiosk_analysis.csv'):
    """여러 CSV 파일을 병합하여 하나의 CSV로 저장합니다."""
    
    # 폴더 내 모든 CSV 파일 경로 가져오기
    csv_files = glob.glob(f"{folder_path}/*.csv")
    
    if not csv_files:
        print(f"오류: '{folder_path}' 폴더에 CSV 파일이 없습니다.")
        return None
    
    # 최대 사용자 수를 추적할 변수
    max_users = 0
    
    # 모든 CSV 파일을 읽고 최대 user_num 추적
    data_frames = []
    for csv_file in csv_files:
        df = pd.read_csv(csv_file)
        
        # 각 데이터프레임에 해당 파일명을 추가
        df['source_file'] = Path(csv_file).name  # 파일명만 추가
        
        # 최대 user_num을 찾기
        if 'user_num' in df.columns:
            max_users = max(max_users, df['user_num'].max())
        
        data_frames.append(df)
    
    print(f"최대 사용자 수: {max_users}명")
    
    # 최대 사용자 수에 맞게 모든 파일의 컬럼을 통일
    for i, df in enumerate(data_frames):
        # 최대 사용자 수보다 적은 열을 null로 채우기
        user_columns = [f'user_{i}_age' for i in range(max_users)] + [f'user_{i}_gender' for i in range(max_users)] + [f'user_{i}_disability' for i in range(max_users)]
        
        # 누락된 열을 추가하고 null로 채우기
        for col in user_columns:
            if col not in df.columns:
                df[col] = None
    
    # 모든 데이터를 합치기
    merged_df = pd.concat(data_frames, ignore_index=True, sort=False)
    
    # 컬럼 순서 정리 (user_num, 나머지 메타데이터, user 데이터)
    all_columns = ['file_name', 'format', 'size', 'width_height', 'environment',
                   'frame_rate', 'total_frames', 'camera_height', 'camera_angle',
                   'InteractionType', 'space_context', 'action_type_0_duration', 
                   'action_type_1_duration', 'action_type_2_duration', 'action_type_3_duration', 'user_num']
    
    user_columns = []
    for i in range(max_users):
        user_columns.extend([f'user_{i}_age', f'user_{i}_gender', f'user_{i}_disability'])
    
    all_columns.extend(user_columns)
    all_columns.append('source_file')  # 마지막에 source_file 추가
    
    # 열 순서 맞추기
    merged_df = merged_df[all_columns]
    
    # CSV로 저장
    output_path = Path(folder_path) / output_file
    merged_df.to_csv(output_path, index=False, encoding='utf-8')
    
    print(f"\n데이터가 성공적으로 병합되어 저장되었습니다: {output_path}")
    
    return merged_df

# 예시 실행
if __name__ == "__main__":
    folder_path = input("CSV 파일들이 있는 폴더 경로를 입력해주세요: ").strip()
    merge_csv_files(folder_path)


최대 사용자 수: 10명


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_125208\1683789386.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_df = pd.concat(data_frames, ignore_index=True, sort=False)



데이터가 성공적으로 병합되어 저장되었습니다: C:\Users\gadi2\Desktop\Downloads\2.라벨링데이터\csv\merged_kiosk_analysis.csv
